Import necessary libs

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.optim.lr_scheduler import StepLR
import joblib
import random
import shap  # For feature importance
import lime  # For feature importance

set device

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load & preprocess data

In [ ]:
# Load data
def load_data(file_path):
    return pd.read_csv(file_path, low_memory=False)

# Convert columns to appropriate types
def convert_column_types(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

#define function that saves a csv matching IDfg with player name
def save_player_names(df):
    player_names = df[['IDfg', 'Name']].drop_duplicates()
    player_names.to_csv('../data/player_names.csv', index=False)

# Filter data for specific seasons, players with sufficient AB, and PA >= 50
def filter_data(df, start_season=2010, min_pa=50):
    df = df[df['Season'] >= start_season]
    df = df[df['PA'] >= min_pa]
    return df

def drop_cols(df):
    df.dropna(axis=1, inplace=True)  # Remove columns with any missing values
    # Drop specific columns
    df.drop(columns=['Age Rng', 'Dol', 'Name'], inplace=True)
    
    # One-hot encode string columns and store the mapping
    string_columns = df.select_dtypes(include=['object']).columns
    one_hot_mapping = {}
    for col in string_columns:
        one_hot_mapping[col] = df[col].unique().tolist()
    df = pd.get_dummies(df, columns=string_columns, drop_first=True)
    
    return df, one_hot_mapping

# Scale both input and target features together (to avoid double scaling)
def scale_features(df, features, scaler=None):
    if scaler is None:
        scaler = StandardScaler()
        df[features] = scaler.fit_transform(df[features])
        joblib.dump(scaler, 'scaler.pkl')  # Save the scaler for future use
    else:
        df[features] = scaler.transform(df[features])
    
    return df, scaler


# Create sequences for model input
def create_sequences(df, input_features, target_features, seq_length=3):
    sequences = []
    grouped_data = df.groupby('IDfg')

    for _, player_data in grouped_data:
        player_data = player_data.sort_values(by='Season')
        input_data = player_data[input_features]
        target_data = player_data[target_features]

        for i in range(len(input_data) - seq_length):
            seq = input_data.iloc[i:i + seq_length].values
            label = target_data.iloc[i + seq_length].values
            sequences.append((seq, label))

    return sequences

# Convert sequences to PyTorch tensors
def to_tensor(sequences):
    X = torch.tensor([s[0] for s in sequences], dtype=torch.float32)
    y = torch.tensor([s[1] for s in sequences], dtype=torch.float32)
    return X, y

# Split the data into train, validation, and test sets
def split_data(sequences, train_ratio=0.7, valid_ratio=0.2):
    train_size = int(len(sequences) * train_ratio)
    valid_size = int(len(sequences) * valid_ratio)
    test_size = len(sequences) - train_size - valid_size

    train_sequences = sequences[:train_size]
    valid_sequences = sequences[train_size:train_size + valid_size]
    test_sequences = sequences[train_size + valid_size:]

    return train_sequences, valid_sequences, test_sequences

def invert_one_hot_encoding(df, one_hot_mapping):
    for col, categories in one_hot_mapping.items():
        # Find the one-hot encoded columns for this original column
        one_hot_cols = [f"{col}_{category}" for category in categories if f"{col}_{category}" in df.columns]
        
        # Create a new column with the original categorical values
        df[col] = df[one_hot_cols].idxmax(axis=1).apply(lambda x: x.split('_', 1)[1])
        
        # Drop the one-hot encoded columns
        df.drop(columns=one_hot_cols, inplace=True)
    
    return df

# Modify preprocess_data to scale features in one step
def preprocess_data(file_path, seq_length=3, start_season=2010, min_pa=50):
    # Load and clean data
    df = load_data(file_path)
    # Save player names
    save_player_names(df)
    df = filter_data(df, start_season=start_season, min_pa=min_pa)
    
    # Drop columns with any missing values and one-hot encode string columns
    df, one_hot_mapping = drop_cols(df)

    # Convert column types
    df = convert_column_types(df)

    # Define input and target features as all columns except 'IDfg' and 'Season'
    input_features = df.columns.difference(['IDfg', 'Season']).tolist()
    target_features = input_features

    # Scale input and target features in one step
    df, scaler = scale_features(df, input_features)

    # Create sequences
    sequences = create_sequences(df, input_features, target_features, seq_length=seq_length)

    # Split sequences into train, validation, and test sets
    train_sequences, valid_sequences, test_sequences = split_data(sequences)

    # Convert to tensors
    X_train, y_train = to_tensor(train_sequences)
    X_valid, y_valid = to_tensor(valid_sequences)
    X_test, y_test = to_tensor(test_sequences)

    # Return preprocessed tensors, scaler, one-hot mapping, and processed DataFrame
    return X_train, y_train, X_valid, y_valid, X_test, y_test, scaler, one_hot_mapping, df

# Preprocess the data
file_path = '../data/mlb_batting_data_2010_2024.csv'
seq_length = 8
X_train, y_train, X_valid, y_valid, X_test, y_test, scaler, one_hot_mapping, df = preprocess_data(file_path, seq_length=seq_length, start_season=2010, min_pa=50)




Train set shapes: X=torch.Size([446, 8, 210]), y=torch.Size([446, 210])
Validation set shapes: X=torch.Size([127, 8, 210]), y=torch.Size([127, 210])
Test set shapes: X=torch.Size([65, 8, 210]), y=torch.Size([65, 210])
Scaled target values (first 5 rows):
       +WPA      -WPA        1B        2B        3B        AB       AVG  \
0  1.653023 -2.135196  1.887219  0.973764 -0.203547  1.820887  0.433267   
1  0.916772 -1.700620  0.947545  0.431230  0.834936  1.445796 -0.337486   
2 -0.563070  0.532700 -0.608789 -0.020882 -0.722788 -0.418627 -0.296920   
3 -0.404079  0.567235 -0.403235 -0.472994 -0.722788 -0.380015 -0.053524   
4 -0.399187  0.311094 -0.403235 -0.111304 -0.722788 -0.363467  0.068173   

       AVG+       Age     BABIP  ...  wSI (pi)  wSI (sc)  wSI/C (pi)  \
0  0.547906  1.227872  0.062676  ... -0.723554 -0.565805   -0.090467   
1 -0.068996  1.483056 -0.811781  ... -1.070153 -1.011411   -0.166014   
2 -0.171813  1.227872 -0.260493  ... -0.376956 -0.518900   -0.458126   
3  0.1

Define model

In [26]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        # Attention mechanism
        self.attention = nn.Linear(hidden_size, 1)
        
        # Batch normalization layer
        self.batch_norm = nn.BatchNorm1d(hidden_size)
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, output_size)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def attention_net(self, lstm_output, final_hidden_state):
        # Applying attention on the hidden state at each time step
        attn_weights = torch.tanh(self.attention(lstm_output))
        attn_weights = torch.softmax(attn_weights, dim=1)

        # Multiply attention weights with the LSTM output to get weighted sum
        context_vector = torch.sum(attn_weights * lstm_output, dim=1)
        return context_vector

    def forward(self, x, lengths):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Pack the padded sequence
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        
        # LSTM forward pass
        packed_output, (hn, cn) = self.lstm(packed_input, (h0, c0))
        
        # Unpack the sequence
        lstm_output, _ = pad_packed_sequence(packed_output, batch_first=True)
        
        # Apply attention mechanism to weigh different time steps
        context_vector = self.attention_net(lstm_output, hn[-1])
        
        # Apply batch normalization
        context_vector = self.batch_norm(context_vector)
        
        # Fully connected layers with dropout
        out = self.fc1(context_vector)
        out = torch.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        
        return out


In [27]:

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=10, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss >= self.best_loss:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [28]:
def collate_fn(batch):
    sequences, targets = zip(*batch)

    # Convert sequences to PyTorch tensors and pad them
    sequences_padded = pad_sequence([seq.clone().detach() for seq in sequences], batch_first=True, padding_value=0)

    # Convert targets to a single tensor
    targets_padded = torch.stack([target.clone().detach() for target in targets])

    # Get lengths
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)

    return sequences_padded, lengths, targets_padded


In [29]:
def print_projected_vs_actual(model, X_train, y_train, target_scaler, num_samples=5):
    """
    Print a comparison between the projected and actual stats for a few samples.

    Args:
        model: Trained model used for prediction.
        X_train: Tensor containing the training feature sequences.
        y_train: Tensor containing the training target values.
        target_scaler: Scaler used to inverse transform the scaled target data.
        num_samples: Number of samples to display for comparison.
    """
    model.eval()
    with torch.no_grad():
        # Select a few random samples to compare
        sample_indices = torch.randint(0, X_train.size(0), (num_samples,))

        for idx in sample_indices:
            # Extract the feature sequence and true target
            seq = X_train[idx].unsqueeze(0).to(device)  # Add batch dimension and move to device
            true_target = y_train[idx].unsqueeze(0).to(device)

            # Predict the stats
            lengths = torch.tensor([seq.size(1)], dtype=torch.int64).cpu()  # Sequence length, move lengths to CPU
            predicted_target = model(seq, lengths)

            # Since predicted_target and true_target are tensors with shape (1, num_features)
            # We need to ensure they match the expected shape for inverse_transform
            predicted_target = predicted_target.squeeze().cpu().numpy()
            true_target = true_target.squeeze().cpu().numpy()

            # Print shapes for debugging
            print(f"Predicted target shape: {predicted_target.shape}")
            print(f"True target shape: {true_target.shape}")
            print(f"Scaler features: {target_scaler.n_features_in_}")

            # Ensure correct shape for scaler
            if predicted_target.shape[0] != target_scaler.n_features_in_:
                raise ValueError(f"Predicted target shape {predicted_target.shape} does not match scaler features {target_scaler.n_features_in_}")

            # Inverse transform the predicted and actual values
            predicted_target = target_scaler.inverse_transform(predicted_target.reshape(1, -1))
            true_target = target_scaler.inverse_transform(true_target.reshape(1, -1))

            # Display the comparison
            print(f"\nSample {idx.item() + 1}:")
            print(f"Actual Stats:    G={true_target[0][0]:.2f}, HR={true_target[0][1]:.2f}, OBP={true_target[0][2]:.3f}, "
                  f"SLG={true_target[0][3]:.3f}, WAR={true_target[0][4]:.2f}")
            print(f"Predicted Stats: G={predicted_target[0][0]:.2f}, HR={predicted_target[0][1]:.2f}, OBP={predicted_target[0][2]:.3f}, "
                  f"SLG={predicted_target[0][3]:.3f}, WAR={predicted_target[0][4]:.2f}")

    model.train()  # Switch back to training mode after evaluation


In [30]:
# Model hyperparameters
input_size = X_train.shape[2]  # Number of features
hidden_size = 128  # Hidden size for complexity
num_layers = 2  # Number of layers (not too deep)
batch_size = 32
output_size = y_train.shape[1]  # Number of target features
dropout = 0.4
criterion = nn.MSELoss()  # Loss function

# Initialize the model
model = LSTM(input_size, hidden_size, num_layers, output_size, dropout).to(device)

# Define optimizer and learning rate scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Learning rate scheduler

# Early stopping setup
early_stopping = EarlyStopping(patience=10, verbose=True)

# Create DataLoader for training and validation
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

valid_dataset = TensorDataset(X_valid, y_valid)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Training loop
num_epochs = 100
print_interval = 1

In [31]:
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    
    for batch_X, lengths, batch_y in train_loader:
        batch_X = batch_X.to(device)  # Move input features to GPU
        batch_y = batch_y.to(device)  # Move targets to GPU
        lengths = lengths.to("cpu")  # Ensure lengths are on CPU for packing/unpacking sequences
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(batch_X, lengths)
        
        # Ensure outputs and targets have matching shapes
        if outputs.shape != batch_y.shape:
            raise ValueError(f"Shape mismatch: outputs shape {outputs.shape}, target shape {batch_y.shape}")
        
        # Loss calculation for multi-target regression
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss:.4f}')
    
    # Validation step
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for val_batch_X, val_lengths, val_batch_y in valid_loader:
            val_batch_X = val_batch_X.to(device)  # Move validation features to GPU
            val_batch_y = val_batch_y.to(device)  # Move validation targets to GPU
            val_lengths = val_lengths.to("cpu")  # Ensure lengths are on CPU for packing/unpacking sequences
            
            val_outputs = model(val_batch_X, val_lengths)
            
            # Ensure validation outputs and targets match in shape
            if val_outputs.shape != val_batch_y.shape:
                raise ValueError(f"Shape mismatch: validation outputs shape {val_outputs.shape}, target shape {val_batch_y.shape}")
            
            val_loss += criterion(val_outputs, val_batch_y).item()
    
    avg_val_loss = val_loss / len(valid_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}')
    
    # Check early stopping
    early_stopping(avg_val_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    # Adjust learning rate
    scheduler.step()
    

# Save the trained model
model_save_path = 'lstm_model.pth'
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')


Epoch 1/100, Training Loss: 0.8697
Epoch 1/100, Validation Loss: 0.8617
Epoch 2/100, Training Loss: 0.7882
Epoch 2/100, Validation Loss: 0.8543
Epoch 3/100, Training Loss: 0.7478
Epoch 3/100, Validation Loss: 0.8076
Epoch 4/100, Training Loss: 0.7270
Epoch 4/100, Validation Loss: 0.8020
Epoch 5/100, Training Loss: 0.6997
Epoch 5/100, Validation Loss: 0.8055
Epoch 6/100, Training Loss: 0.7058
Epoch 6/100, Validation Loss: 0.9118
Epoch 7/100, Training Loss: 0.7019
Epoch 7/100, Validation Loss: 0.8255
Epoch 8/100, Training Loss: 0.6752
Epoch 8/100, Validation Loss: 0.8515
Epoch 9/100, Training Loss: 0.6764
Epoch 9/100, Validation Loss: 0.8185
Epoch 10/100, Training Loss: 0.6587
Epoch 10/100, Validation Loss: 0.8077
Epoch 11/100, Training Loss: 0.6355
Epoch 11/100, Validation Loss: 0.8000
Epoch 12/100, Training Loss: 0.6243
Epoch 12/100, Validation Loss: 0.8192
Epoch 13/100, Training Loss: 0.6182
Epoch 13/100, Validation Loss: 0.8180
Epoch 14/100, Training Loss: 0.6080
Epoch 14/100, Valida

Test on individual players

In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np

# Define the manual unscale function
def manual_unscale(y_scaled, scaler, target_feature_indices=None):
    if target_feature_indices is not None:
        # Extract the relevant mean and scale for the target columns
        target_means = scaler.mean_[target_feature_indices]
        target_scales = scaler.scale_[target_feature_indices]

        # Manually apply inverse transformation
        y_unscaled = (y_scaled * target_scales) + target_means
        return y_unscaled

    return scaler.inverse_transform(y_scaled)

# Function to choose a random player
def choose_random_player(df):
    eligible_players = df[df['Season'] == 2023].groupby('IDfg').filter(lambda x: x['AB'].sum() >= 100)['IDfg'].unique()
    return np.random.choice(eligible_players)

# Function to gather all years of player data
def gather_player_data(df, player_id):
    player_data = df[df['IDfg'] == player_id].sort_values(by='Season')
    return player_data

# Function to predict future stats
def predict_future_stats(player_id, input_features, target_features, model, scaler, df, one_hot_mapping):
    # Gather all years of player data
    player_data = gather_player_data(df, player_id)
    if player_data.empty:
        print(f"No data found for player ID {player_id}.")
        return

    last_known_data = player_data[input_features].iloc[-1].values
    player_name = player_data['Name'].iloc[0]  # Assuming 'Name' column exists

    # Print player's historical data
    print(f"\nPlayer: {player_name}\n")
    print(f"Previous stats:")
    for _, row in player_data.iterrows():
        print(f"Year {row['Season']} - Age {row['Age']}: G: {row['G']}, HR: {row['HR']}, OBP: {row['OBP']:.3f}, SLG: {row['SLG']:.3f}, WAR: {row['WAR']:.3f}")

    predictions = []

    print("\nPredicted future stats (2024-2026):")
    for year in range(2024, 2027):
        # Calculate the correct age
        current_age = last_known_data[0] + 1
        
        # Prepare input data for the model
        input_data = last_known_data.reshape(1, -1)

        # Scale the input data
        scaled_input = scaler.transform(input_data)

        # Convert to tensor and predict
        input_tensor = torch.tensor(scaled_input, dtype=torch.float32).to(device)
        predicted_stats = model(input_tensor.unsqueeze(1), torch.tensor([1], dtype=torch.int64).cpu())
        predicted_stats = predicted_stats.squeeze().cpu().detach().numpy()

        # Ensure the predictions are in the right shape for inverse scaling
        predicted_stats_reshaped = predicted_stats.reshape(1, -1)

        # Manually unscale the predictions
        target_feature_indices = [input_features.index(f) for f in target_features]
        unscaled_predictions = manual_unscale(predicted_stats_reshaped, scaler, target_feature_indices)

        # Replace the predicted age with the correct age
        unscaled_predictions[0][0] = current_age

        # Store the predictions
        predictions.append({
            'Year': year,
            'Age': current_age,
            'G': unscaled_predictions[0][1],
            'HR': unscaled_predictions[0][2],
            'OBP': unscaled_predictions[0][3],
            'SLG': unscaled_predictions[0][4],
            'WAR': unscaled_predictions[0][5]
        })
        
        # Update last_known_data for the next iteration
        last_known_data = np.array([
            current_age,
            unscaled_predictions[0][1],
            unscaled_predictions[0][2],
            unscaled_predictions[0][3],
            unscaled_predictions[0][4],
            unscaled_predictions[0][5]  # Update WAR with the predicted value
        ])

    # Print the predictions
    for prediction in predictions:
        print(f"Year {prediction['Year']} - Age {prediction['Age']}:")
        print(f"G: {prediction['G']:.2f}, HR: {prediction['HR']:.2f}, OBP: {prediction['OBP']:.3f}, SLG: {prediction['SLG']:.3f}, WAR: {prediction['WAR']:.3f}\n")
# Preprocess the data
file_path = '../data/mlb_batting_data_2010_2024.csv'
seq_length = 8
X_train, y_train, X_valid, y_valid, X_test, y_test, scaler, one_hot_mapping, processed_df = preprocess_data(file_path, seq_length=seq_length, start_season=2010, min_pa=50)

# Print shapes for debugging
print(f"Train set shapes: X={X_train.shape}, y={y_train.shape}")
print(f"Validation set shapes: X={X_valid.shape}, y={y_valid.shape}")
print(f"Test set shapes: X={X_test.shape}, y={y_test.shape}")

# Get a random player
player_id = choose_random_player(processed_df)

# Define input and target features
input_features = processed_df.columns.difference(['IDfg', 'Season']).tolist()
target_features = input_features

# Load scaler
scaler = joblib.load('scaler.pkl')
print(f"Scaler mean: {scaler.mean_}, scale: {scaler.scale_}")

# Predict future stats for the chosen player
predict_future_stats(player_id, input_features, target_features, model, scaler, processed_df, one_hot_mapping)

FileNotFoundError: [Errno 2] No such file or directory: '../mlb_batting_data_1912_2023.csv'